### This code was written solely by Amrit

In [ ]:
import numpy as np
%matplotlib inline
from pyqubo import Array, Placeholder, Constraint
import matplotlib.pyplot as plt
import networkx as nx
import neal
import csv
import os
import math
import pandas as pd

In [ ]:
def parse_tsp_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    node_coord_section = False
    node_matrix = []
    distance_matrix = None

    for line in lines:
        if line.startswith("NODE_COORD_SECTION"):
            node_coord_section = True
            continue
        elif line.startswith("EOF"):
            break

        if node_coord_section:
            node_info = line.split()
            node_id = int(node_info[0])
            # x_coord = int(float(node_info[1]))  # Convert to int
            # y_coord = int(float(node_info[2]))  # Convert to int
            x_coord = float(node_info[1])  # Convert to int
            y_coord = float(node_info[2])  # Convert to int

            node_matrix.append([node_id, x_coord, y_coord])

    node_matrix = np.array(node_matrix)

    # Calculate distance matrix (as in the previous version)
    num_nodes = node_matrix.shape[0]
    distance_matrix = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            x1, y1 = node_matrix[i, 1], node_matrix[i, 2]
            x2, y2 = node_matrix[j, 1], node_matrix[j, 2]
            distance_matrix[i, j] = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    return node_matrix, distance_matrix

In [ ]:
def plot_city(cities, sol=None):
    n_city = len(cities)
    cities_dict = dict(cities)
    G = nx.Graph()
    for city in cities_dict:
        G.add_node(city)

    # draw path
    if sol:
        city_order = []
        for i in range(n_city):
            for j in range(n_city):
                if sol.array('c', (i, j)) == 1:
                    city_order.append(j)
        for i in range(n_city):
            city_index1 = city_order[i]
            city_index2 = city_order[(i+1) % n_city]
            G.add_edge(cities[city_index1][0], cities[city_index2][0])

    plt.figure(figsize=(3,3))
    pos = nx.spring_layout(G)
    nx.draw_networkx(G, cities_dict)
    plt.axis("off")
    plt.show()

def dist(i, j, cities):
    pos_i = cities[i][1]
    pos_j = cities[j][1]
    return np.sqrt((pos_i[0] - pos_j[0])**2 + (pos_i[1] - pos_j[1])**2)

In [ ]:
def process_tsp_files_in_folder(folder_name, num_cities_in_a_instance=30):
    if not os.path.exists(folder_name):
        raise ValueError(f"Folder '{folder_name}' does not exist.")

    files_in_folder = [file for file in os.listdir(folder_name) if os.path.isfile(os.path.join(folder_name, file))]

    for file_name in files_in_folder:
        if file_name.endswith('.tsp'):
            file_path = os.path.join(folder_name, file_name)

    return file_path

In [ ]:
def solve_for_instances(file_path, nodes_matrix, relaxation_parameter, num_cities_in_a_instance=30):
    cities = [(i,(0,0)) for i in range(num_cities_in_a_instance)]
    n_mat = nodes_matrix.copy()
    for i in range(num_cities_in_a_instance):
        cities[i] = (str(n_mat[i][0]), (n_mat[i][1], n_mat[i][2]))
    #plot_city(cities)
    #print(cities)

    ## Prepare binary vector with bit (𝑖,𝑗) representing to visit 𝑗 city at time 𝑖
    n_city = len(cities)
    x = Array.create('c', (n_city, n_city), 'BINARY')
    # print(x)

    ## Constraint not to visit more than two cities at the same time.
    ## equation (6) implemented here
    ## time_const + city_const = H_a
    time_const = 0.0
    for i in range(n_city):
        # If you wrap the hamiltonian by Const(...), this part is recognized as constraint
        time_const += Constraint((sum(x[i, j] for j in range(n_city)) - 1)**2, label="time{}".format(i))

    ## Constraint not to visit the same city more than twice.
    city_const = 0.0
    for j in range(n_city):
        city_const += Constraint((sum(x[i, j] for i in range(n_city)) - 1)**2, label="city{}".format(j))

    # print("time_const: ", time_const)
    # print("city_const: ", city_const)

    ## distance of route
    ## equation (5) implemented here
    distance = 0.0
    for i in range(n_city):
        for j in range(n_city):
            for k in range(n_city):
                d_ij = dist(i, j, cities)
                distance += d_ij * x[k, i] * x[(k+1)%n_city, j]  # sum(d_uv) in eq.5 i.e. H_b

    # print("distance: ", distance)

    ## Construct hamiltonian
    A = Placeholder("A")  # the relaxation parameter
    H = distance + A * (time_const + city_const)  # Eq (4)

    # print("Relaxation Parameter A: ", A)
    # print("Hamiltonian H: ", H)

    ## Compile model
    model = H.compile()

    ## Generate QUBO
    ## maybe we can add a loop to go through different relaxation parameter here
    feed_dict = {'A': relaxation_parameter}  # setting it to upper bound of the coordinate works!!!
    bqm = model.to_bqm(feed_dict=feed_dict)


    sa = neal.SimulatedAnnealingSampler()
    sampleset = sa.sample(bqm, num_reads=128, num_sweeps=100)

    # Decode solution
    decoded_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
    best_sample = min(decoded_samples, key=lambda x: x.energy)
    energy = best_sample.energy
    num_broken = len(best_sample.constraints(only_broken=True))
    infeasible_ctr = 0
    if num_broken == 0:
        #print(energy)
        infeasible_ctr = 0
    for i in range(128):
        if len(decoded_samples[i].constraints(only_broken=True)) > 0:
            infeasible_ctr += 1
    P_f = (128-infeasible_ctr)/128      #probability of feasibility


    return P_f, energy, decoded_samples

In [ ]:
def param_bin_search(file_name, file_path):
    nodes_matrix, distance_matrix = parse_tsp_file(file_path)
    lower_bound = 0
    upper_bound = 10000
    mid = 0
    lb_found = False
    ub_found = False
    log_scale = [10**i for i in range(7)]

    # Search in Log scale
    for i in log_scale:
        p_f, energy, decoded_samples = solve_for_instances(file_path,nodes_matrix,i)
        print(i,'->',p_f)
        if p_f > 0 and p_f <= 0.1:
            lower_bound = i
            lb_found = True
            break
        elif p_f == 0:
            lower_bound= 0
        elif p_f > 0.9 and p_f < 1 :
            upper_bound = i
            ub_found=True
            break
        elif p_f == 1:
            upper_bound = i
            break

    lower_bound = 0
    # Search for upper bound
    while lower_bound<=upper_bound and not ub_found:
        mid = (upper_bound + lower_bound)//2
        p_f, energy, decoded_samples = solve_for_instances(file_path,nodes_matrix,mid)
        print(mid,"->", p_f)
        if p_f == 0 :
            lower_bound = mid
        elif p_f ==1:
            upper_bound = mid
        elif p_f < 1 and p_f >= 0.9:
            upper_bound = mid
            ub_found = True
        else:
            lower_bound = mid

    upper_bound1 = upper_bound
    lower_bound = 0
    # Search for lower bound
    while lower_bound<=upper_bound and not lb_found :
        mid = (upper_bound + lower_bound)//2
        p_f, energy, decoded_samples = solve_for_instances(file_path,nodes_matrix,mid)
        print(mid, "->", p_f)
        if p_f ==0 :
            lower_bound = mid
        elif p_f ==1:
            upper_bound = mid
        elif p_f > 0 and p_f <= 0.1:
            lower_bound = mid
            lb_found = True
        else:
            upper_bound = mid

    return upper_bound1, lower_bound

In [ ]:
def energies(decoded_samples):
    energies = [0 for i in range(128)]
    for i in range(128):
        energies[i] = decoded_samples[i].energy
    E_avg = sum(energies)/128
    s = 0
    for i in range(128):
        s = s + (E_avg-energies[i])**2
    E_std = math.sqrt((s/128))
    return E_avg, E_std

In [ ]:
# Searching for the range and saving results in pandas dataframe and CSV file
def FindParameterBoundaries(folder_name):
    folder_name = "TSPFiles"
    files_in_folder = [file for file in os.listdir(folder_name) if os.path.isfile(os.path.join(folder_name, file))]
    results_dict = {}

    for file_name in files_in_folder:
        if file_name.endswith('.tsp'):
            file_path = os.path.join(folder_name, file_name)
            print(f"Finding Parameter for {file_name}")
            print(file_path)
            ub, lb = param_bin_search(file_name, file_path)
            results_dict.update({file_name:(ub,lb)})
            print(f"Parameter range for {file_name} are {ub} and {lb}")

    column_names = ["File_name","Upper_bound","Lower_Bound"]
    range_df = pd.DataFrame(columns=column_names)
    for file in files_in_folder:
        temp_list = [file, results_dict[file][0], results_dict[file][1]]
        range_df.loc[len(range_df)] = temp_list
    range_df.to_csv("ParameterBoundaryResults/parameter_range.csv")


In [ ]:
FindParameterBoundaries("TSPFiles")

Finding Parameter for berlin52.tsp
TSPFiles\berlin52.tsp
1 -> 0.0
10 -> 0.0
100 -> 0.0
1000 -> 0.7109375
10000 -> 1.0
5000 -> 1.0
2500 -> 1.0
1250 -> 0.953125
625 -> 0.0546875
Parameter range for berlin52.tsp are 1250 and 625
Finding Parameter for bier127.tsp
TSPFiles\bier127.tsp
1 -> 0.0
10 -> 0.0
100 -> 0.0
1000 -> 0.0
10000 -> 1.0
5000 -> 1.0
2500 -> 0.3359375
3750 -> 0.8359375
4375 -> 0.9609375
2187 -> 0.2265625
1093 -> 0.0
1640 -> 0.0
1913 -> 0.03125
Parameter range for bier127.tsp are 4375 and 1913
Finding Parameter for brd14051.tsp
TSPFiles\brd14051.tsp
1 -> 0.0
10 -> 0.0
100 -> 0.0
1000 -> 0.625
10000 -> 1.0
5000 -> 1.0
2500 -> 1.0
1250 -> 0.7109375
1875 -> 1.0
1562 -> 0.9765625
781 -> 0.421875
390 -> 0.0
585 -> 0.2109375
487 -> 0.0
536 -> 0.046875
Parameter range for brd14051.tsp are 1562 and 536
Finding Parameter for ch130.tsp
TSPFiles\ch130.tsp
1 -> 0.0
10 -> 0.0
100 -> 0.0
1000 -> 1.0
500 -> 0.828125
750 -> 1.0
625 -> 0.9765625
312 -> 0.03125
Parameter range for ch130.tsp a